In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

import MCCoordinator
import StorageSampler

def storage_system(**kwargs):
    """
    Instantiate StorageSystem object using GB system data

    :param wind_power: assumed wind power capacity (in MW)
    :param kwargs: additional arguments to be supplied to the StorageSystem constructor
    :return: StorageSystem object
    """
    import pandas as pd
    import os
    data = pd.read_csv('../data/UKdata/20161213_uk_wind_solar_demand_temperature.csv',
                       parse_dates=['UTC Time', 'Local Time'], infer_datetime_format=True, dayfirst=True, index_col=0)

    demand_data = data['demand_net'].dropna()['2006':'2015']
    wind_data = 10000 * data['wind_merra1'].dropna()

    demand_samples = {yeardata[0]: yeardata[1].values[:8760] for yeardata in
                      demand_data.groupby(demand_data.index.year)}
    wind_samples = {yeardata[0]: yeardata[1].values[:8760] for yeardata in wind_data.groupby(wind_data.index.year)}

    dataframe = pd.read_csv('../data/UKdata/battery_data.csv')
    store_power_list=3*dataframe['Power (MW)'][0:27]
    store_energy_list=3*dataframe['Energy (MWh)'][0:27]

    return StorageSampler.StorageSystem(demand_samples=demand_samples, 
                                        wind_samples=wind_samples, 
                                        store_power_list=store_power_list,
                                        store_energy_list=store_energy_list,
                                        **kwargs)


system = storage_system()

import MachineLearning
system.AI_model = MachineLearning.MachineLearning(train_size=500)

/var/folders/pr/b8ltbq5x39n0gw92v8zlc8q00000gn/T/ipykernel_17816/1315386803.py:19: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  data = pd.read_csv('../data/UKdata/20161213_uk_wind_solar_demand_temperature.csv',


Generator set: [1200, 600, 600, 250, 250, 120, 60, 20, 20, 10, 10, 10]
Optimising number of generator sets to LOLE target of 3 hours
20 generator sets. Base LOLE=2.92 h; determined load offset of 36.15 MW to reach LOLE target.



In [2]:
def pool_generation(system, pool_size):
    pool_temporal = np.zeros((pool_size, 24))

    for i in range(pool_size):
        pool_temporal[i] = system.generate_daily_margin_trace()
    return pool_temporal

def load_data(file_name):
    '''
    load data from the file_name
    Parameters: 
        file_name: string
            file address
    Returns:
        data: ndarray
            an array of file's content
    '''
    data = np.ascontiguousarray(np.genfromtxt(file_name, delimiter=','))
    return data

In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error

from modAL.models import ActiveLearner

# Active learning strategy
def random_forest_regression_std(regressor, X, n_instances=1):
    all_tree_preds = np.array([tree.predict(X) for tree in regressor.estimator.estimators_])  # Shape: (n_trees, n_samples)

    std = np.var(all_tree_preds, axis=0)
    
    query_idx = np.argsort(std)[-n_instances:]  # Last 10 indices after sorting
    return query_idx, X[query_idx]

def random_forest_regression_std_analysis(estimator, X, n_instances=1):
    all_tree_preds = np.array([tree.predict(X) for tree in estimator.estimators_])  # Shape: (n_trees, n_samples)

    std = np.var(all_tree_preds, axis=0)
    
    query_idx = np.argsort(std)[-n_instances:]  # Last 10 indices after sorting
    return query_idx, X[query_idx], std[query_idx]

# Random sampling strategy
def random_sampling(regressor, X, n_instances=1):
    query_idx = np.random.choice(X.shape[0], n_instances, replace=False)

    return query_idx, X[query_idx]

In [4]:
# LOL and ENS surrogate model for a year
def surrogate_model_year_prediction(estimator, margin_year):
    margin_daily = margin_year.reshape(-1, 365, 24)
    return [np.sum(estimator.predict(margin_daily[i,:, :])) for i in range(margin_year.shape[0])]

In [5]:
def surrogate_model_training(system, training_strategy, n_train_random=1825, n_initial=1825, n_queries=20, temporal_pool_size=3650, n_instances=91):
    time_start = time.time()
    if training_strategy == 'Random':
        # Generate the training data
        X_train = pool_generation(system, n_train_random)
        y_train = np.array([system.run_optimal_policy(X_train[i]) for i in range(X_train.shape[0])])

        lol_train = y_train[:,0]
        ens_train = y_train[:,1]

        lol_learner = RandomForestRegressor().fit(X_train, lol_train)
        ens_learner = RandomForestRegressor().fit(X_train, ens_train)
    
    elif training_strategy == 'AL':    
        X_train = pool_generation(system, n_initial)

        y_train = np.array([system.run_optimal_policy(X_train[i]) for i in range(X_train.shape[0])])
        lol_train = y_train[:,0]
        ens_train = y_train[:,1]

        # Initialize the learner
        lol_learner = ActiveLearner(
            estimator=RandomForestRegressor(),
            query_strategy=random_forest_regression_std,
            X_training=X_train, y_training=lol_train
        )

        ens_learner = ActiveLearner(
            estimator=RandomForestRegressor(),
            query_strategy=random_forest_regression_std,
            X_training=X_train, y_training=ens_train
        )

        for i in range(n_queries):
            X_pool_temporal = pool_generation(system, pool_size=temporal_pool_size)
            
            query_idx, query_inst = ens_learner.query(X_pool_temporal, n_instances=n_instances)

            y_new = np.array([system.run_optimal_policy(query_inst[j]) for j in range(query_inst.shape[0])])
            
            lol_learner.teach(query_inst, y_new[:,0])
            ens_learner.teach(query_inst, y_new[:,1])
    else:
        raise ValueError('Unknown training strategy: {}'.format(training_strategy))
        
    time_end = time.time()

    return lol_learner, ens_learner, time_end - time_start

In [6]:
def run_MLMC(system, samples, n_run, time_seconds, ml_hierarchy, use_joblib, verbose):
    '''
    Run MLMC simulation
    Parameters:
        samples: int
            initial number of samples in each level for sigma estimation.
        n_run: int
            number of runs for MLMC simulation.
        time_seconds: int
            duration for each run of MLMC simulation.
        ml_hierarchy: {'OptimalNStore', 'GreedyNStore', 'AIGreedyNStore', 'Greedy1Store', 'AvgStore', 'NoStore', 'AIModel'}
            set of models for MLMC structure.
        use_joblib: bool
            if Ture: use all cores, otherwise run on a single core.
        Verbose: bool
            if Ture: print with details, otherwise: print summery of results
    '''
    mcc = MCCoordinator.MCCoordinator(factory=system, 
                                    ml_hierarchy=ml_hierarchy, 
                                    use_expectations=True, 
                                    use_joblib=use_joblib, joblib_n_jobs=-1, joblib_batch_size=5)
    mcc.explore(n_samples=samples)
    for i in range(n_run):
        mcc.run_recommended(time_seconds=time_seconds, verbose=verbose, optimization_target='EENS')
    mcc.verbose_result()
    return mcc


def round_to_n(x, n):
    """
    Round x to n significant digits.

    :param x: scalar to be converted
    :param n: number of significant digits
    :return: scalar
    """
    import math

    if np.isnan(x): return x
    if np.sign(x) == 0: return 0.0
    return np.round(x, -int(math.floor(math.log10(abs(x)))) + (n - 1))


def generate_mcc_results(mcc):
    """
    Print summary results to screen.

    :return: None
    """
    # Suppress divide by zeros and nan errors; store old settings to restore on return.
    old_settings = np.seterr(divide='ignore', invalid='ignore')

    # Compute the total time as computed by index computations and sample computations
    total_time_level = sum([level_stats.sum for level_stats in mcc.level_exec_times.values()])
    total_time = sum([set_stats.sum for set_stats in mcc.set_exec_times.values()])

    # compute the overall MLMC estimate and its standard error, by quadratic summation of term errors
    mean_result = sum([ml_stats.mean for key, ml_stats in mcc.ml_stats.items()])
    stderr_result = np.sqrt(sum([ml_stats.stderr ** 2 for key, ml_stats in mcc.ml_stats.items()]))

    est_time_per_target_sample = min([mcc.set_exec_times[level_set].mean for level_set in mcc.set_exec_times.keys() if mcc.target_level in level_set])
    target_stats = mcc.level_stats[mcc.target_level]
    est_time_spent_for_target_result = target_stats.count * est_time_per_target_sample

    # report results, alongside an estimated 'computational speed'
    results = [round_to_n(mcc.wall_clock_time, 3)]  # Simulation time
    for result_index, result_label in enumerate(mcc.output_labels):
        results.append(round_to_n(mean_result[result_index], 4)) # Estimation of the result (LOL or ENS)
        results.append(round_to_n(stderr_result[result_index], 2)) # Standard error of the result (LOL or ENS)
        results.append(round_to_n(mean_result[result_index] ** 2 / (stderr_result[result_index] ** 2 * mcc.wall_clock_time), 3)) # Computational speed

    for idx_set, set_stats in mcc.set_exec_times.items():
        # note: include 'set' to remove 'frozenset' bracket
        results.append(set_stats.count) # number of samples in each level
        results.append(round_to_n(set_stats.mean, 6)) # average exec time at each level

    for idx_set, ml_stats in mcc.ml_stats.items():
        level_mean_result = ml_stats.mean
        level_stderr_result = ml_stats.stderr
        for result_index, result_label in enumerate(mcc.output_labels):
            results.append(round_to_n(level_mean_result[result_index], 4)) # contribution in estimation
            results.append(round_to_n(level_stderr_result[result_index], 2)) # contribution in std

    return results

# Active learning

In [7]:
# Number of times to run MLMC
n_run_MLMC = 1

# initialize parameters for run
samples = 20
n_run = 40
time_seconds = 50
training_strategy = 'AL' # 'AL' or 'Random'

######################################################################################################
# Hyperparameters for active learning
n_initial = 365 * 2 # Initial training set, 2 years
n_queries = 10 # 10 years
temporal_pool_size = 3650 # 10 years
n_instances = 91 # 1 season = 91 days

#####################################################################################################
# Hyperparameters for random sampling
n_train_random = 365 * 5 + 365 * 5 * 18 # Initial training set, 5 years

# Estimator: Surr , Architecture: Exact|Surrogate models
ml_hierarchy = ['OptimalNStore', 'AIModel']

# [
#  "Training time", "Simulation time", "mean_LOL", "stderr_LOL", "speed_LOL", "mean_ENS", "stderr_ENS", "speed_ENS",
#  "Exact model calls", "exact model time per run", "Surrogate model calls", "Surrogate model time per run",
#  "exact_contribution_LOL_mean", "exact_contribution_LOL_std", "surrogate_contribution_LOL_mean", "surrogate_contribution_LOL_std",
#  "exact_contribution_ENS_mean", "exact_contribution_ENS_std", "surrogate_contribution_ENS_mean", "surrogate_contribution_ENS_std",
# ]
results = np.zeros((n_run_MLMC, 20))

print('Training strategy: ', training_strategy)
print('n_train_random: ', n_train_random)

for i in range(n_run_MLMC):
    lol_learner, ens_learner, results[i,0] = surrogate_model_training(system, training_strategy, n_train_random, n_initial, n_queries, temporal_pool_size, n_instances)
    system.AI_model.lol_model = lol_learner
    system.AI_model.ens_model = ens_learner

    mcc = run_MLMC(system, samples, n_run, time_seconds, ml_hierarchy, use_joblib=False, verbose=False)
    results[i,1:] = generate_mcc_results(mcc)

avg_results = np.mean(results, axis=0)
print('Average results: ', avg_results)

Training strategy:  AL
n_train_random:  34675


/Users/rookierq/opt/anaconda3/envs/MLMC_AL/lib/python3.9/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/rookierq/opt/anaconda3/envs/MLMC_AL/lib/python3.9/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/rookierq/opt/anaconda3/envs/MLMC_AL/lib/python3.9/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/rookierq/opt/anaconda3/envs/MLMC_AL/lib/python3.9/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/rookierq/opt/anaconda3/envs/MLMC_AL/lib/python3.9/site-packages/sklearn/utils/deprecation

Running: {frozenset({'OptimalNStore', 'AIModel'}): 20, frozenset({'AIModel'}): 20}
relative sample numbers:  {frozenset({'OptimalNStore', 'AIModel'}): 1484.7973453954087, frozenset({'AIModel'}): 65993.00252862377}
Running: {frozenset({'OptimalNStore', 'AIModel'}): 26, frozenset({'AIModel'}): 1156}
relative sample numbers:  {frozenset({'OptimalNStore', 'AIModel'}): 2210.4259595887397, frozenset({'AIModel'}): 72665.71512807813}
Running: {frozenset({'OptimalNStore', 'AIModel'}): 28, frozenset({'AIModel'}): 940}
relative sample numbers:  {frozenset({'OptimalNStore', 'AIModel'}): 2287.0144561383254, frozenset({'AIModel'}): 74296.297199433}
Running: {frozenset({'OptimalNStore', 'AIModel'}): 28, frozenset({'AIModel'}): 937}
relative sample numbers:  {frozenset({'OptimalNStore', 'AIModel'}): 2839.2326157629946, frozenset({'AIModel'}): 90874.61967012341}
Running: {frozenset({'OptimalNStore', 'AIModel'}): 29, frozenset({'AIModel'}): 930}
relative sample numbers:  {frozenset({'OptimalNStore', 'AI

In [8]:
# Number of times to run MLMC
n_run_MLMC = 1

# initialize parameters for run
samples = 20
n_run = 40
time_seconds = 50
training_strategy = 'AL' # 'AL' or 'Random'

######################################################################################################
# Hyperparameters for active learning
n_initial = 365 * 2 # Initial training set, 5 years
n_queries = 20 # 10 years
temporal_pool_size = 3650 # 10 years
n_instances = 91 # 1 season = 91 days

################################################################################################
# Hyperparameters for random sampling
n_train_random = 2735  # 365 * 5 + 365 * 5 * 5 # Initial training set, 5 years


# Estimator: Surr , Architecture: Exact|Surrogate models
ml_hierarchy = ['OptimalNStore', 'AIModel']

# [
#  "Training time", "Simulation time", "mean_LOL", "stderr_LOL", "speed_LOL", "mean_ENS", "stderr_ENS", "speed_ENS",
#  "Exact model calls", "exact model time per run", "Surrogate model calls", "Surrogate model time per run",
#  "exact_contribution_LOL_mean", "exact_contribution_LOL_std", "surrogate_contribution_LOL_mean", "surrogate_contribution_LOL_std",
#  "exact_contribution_ENS_mean", "exact_contribution_ENS_std", "surrogate_contribution_ENS_mean", "surrogate_contribution_ENS_std",
# ]
results = np.zeros((n_run_MLMC, 20))

print('Training strategy: ', training_strategy)
print('n_train_random: ', n_train_random)

for i in range(n_run_MLMC):
    lol_learner, ens_learner, results[i,0] = surrogate_model_training(system, training_strategy, n_train_random, n_initial, n_queries, temporal_pool_size, n_instances)
    system.AI_model.lol_model = lol_learner
    system.AI_model.ens_model = ens_learner

    mcc = run_MLMC(system, samples, n_run, time_seconds, ml_hierarchy, use_joblib=False, verbose=False)
    results[i,1:] = generate_mcc_results(mcc)

avg_results = np.mean(results, axis=0)
print('Average results: ', avg_results)

Training strategy:  AL
n_train_random:  2735


/Users/rookierq/opt/anaconda3/envs/MLMC_AL/lib/python3.9/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/rookierq/opt/anaconda3/envs/MLMC_AL/lib/python3.9/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/rookierq/opt/anaconda3/envs/MLMC_AL/lib/python3.9/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/rookierq/opt/anaconda3/envs/MLMC_AL/lib/python3.9/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/rookierq/opt/anaconda3/envs/MLMC_AL/lib/python3.9/site-packages/sklearn/utils/deprecation

Running: {frozenset({'OptimalNStore', 'AIModel'}): 20, frozenset({'AIModel'}): 20}
relative sample numbers:  {frozenset({'OptimalNStore', 'AIModel'}): 1984.392605845524, frozenset({'AIModel'}): 88537.00414526406}
Running: {frozenset({'OptimalNStore', 'AIModel'}): 25, frozenset({'AIModel'}): 1147}
relative sample numbers:  {frozenset({'OptimalNStore', 'AIModel'}): 2011.213958039719, frozenset({'AIModel'}): 65903.2478341649}
Running: {frozenset({'OptimalNStore', 'AIModel'}): 28, frozenset({'AIModel'}): 939}
relative sample numbers:  {frozenset({'OptimalNStore', 'AIModel'}): 2337.248793244967, frozenset({'AIModel'}): 74993.03841792316}
Running: {frozenset({'OptimalNStore', 'AIModel'}): 28, frozenset({'AIModel'}): 924}
relative sample numbers:  {frozenset({'OptimalNStore', 'AIModel'}): 2705.4527794992264, frozenset({'AIModel'}): 86692.37876363115}
Running: {frozenset({'OptimalNStore', 'AIModel'}): 28, frozenset({'AIModel'}): 918}
relative sample numbers:  {frozenset({'OptimalNStore', 'AIMo

In [9]:
# Number of times to run MLMC
n_run_MLMC = 1

# initialize parameters for run
samples = 20
n_run = 40
time_seconds = 50
training_strategy = 'AL' # 'AL' or 'Random'

######################################################################################################
# Hyperparameters for active learning
n_initial = 365 * 2 # Initial training set, 5 years
n_queries = 5 # 10 years
temporal_pool_size = 3650 # 10 years
n_instances = 91 # 1 season = 91 days

################################################################################################
# Hyperparameters for random sampling
n_train_random = 2735  # 365 * 5 + 365 * 5 * 5 # Initial training set, 5 years


# Estimator: Surr , Architecture: Exact|Surrogate models
ml_hierarchy = ['OptimalNStore', 'AIModel']

# [
#  "Training time", "Simulation time", "mean_LOL", "stderr_LOL", "speed_LOL", "mean_ENS", "stderr_ENS", "speed_ENS",
#  "Exact model calls", "exact model time per run", "Surrogate model calls", "Surrogate model time per run",
#  "exact_contribution_LOL_mean", "exact_contribution_LOL_std", "surrogate_contribution_LOL_mean", "surrogate_contribution_LOL_std",
#  "exact_contribution_ENS_mean", "exact_contribution_ENS_std", "surrogate_contribution_ENS_mean", "surrogate_contribution_ENS_std",
# ]
results = np.zeros((n_run_MLMC, 20))

print('Training strategy: ', training_strategy)
print('n_train_random: ', n_train_random)

for i in range(n_run_MLMC):
    lol_learner, ens_learner, results[i,0] = surrogate_model_training(system, training_strategy, n_train_random, n_initial, n_queries, temporal_pool_size, n_instances)
    system.AI_model.lol_model = lol_learner
    system.AI_model.ens_model = ens_learner

    mcc = run_MLMC(system, samples, n_run, time_seconds, ml_hierarchy, use_joblib=False, verbose=False)
    results[i,1:] = generate_mcc_results(mcc)

avg_results = np.mean(results, axis=0)
print('Average results: ', avg_results)

Training strategy:  AL
n_train_random:  2735


/Users/rookierq/opt/anaconda3/envs/MLMC_AL/lib/python3.9/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/rookierq/opt/anaconda3/envs/MLMC_AL/lib/python3.9/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/rookierq/opt/anaconda3/envs/MLMC_AL/lib/python3.9/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/rookierq/opt/anaconda3/envs/MLMC_AL/lib/python3.9/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/rookierq/opt/anaconda3/envs/MLMC_AL/lib/python3.9/site-packages/sklearn/utils/deprecation

Running: {frozenset({'OptimalNStore', 'AIModel'}): 20, frozenset({'AIModel'}): 20}
relative sample numbers:  {frozenset({'OptimalNStore', 'AIModel'}): 1698.8300731457707, frozenset({'AIModel'}): 62005.99907381078}
Running: {frozenset({'OptimalNStore', 'AIModel'}): 25, frozenset({'AIModel'}): 937}
relative sample numbers:  {frozenset({'OptimalNStore', 'AIModel'}): 5863.413160412177, frozenset({'AIModel'}): 138542.6977213342}
Running: {frozenset({'OptimalNStore', 'AIModel'}): 31, frozenset({'AIModel'}): 750}
relative sample numbers:  {frozenset({'OptimalNStore', 'AIModel'}): 4538.413110454783, frozenset({'AIModel'}): 110768.36279277285}
Running: {frozenset({'OptimalNStore', 'AIModel'}): 31, frozenset({'AIModel'}): 772}
relative sample numbers:  {frozenset({'OptimalNStore', 'AIModel'}): 3831.2102406729696, frozenset({'AIModel'}): 94426.59854333862}
Running: {frozenset({'OptimalNStore', 'AIModel'}): 31, frozenset({'AIModel'}): 777}
relative sample numbers:  {frozenset({'OptimalNStore', 'AI

# Random sampling

In [10]:
# Number of times to run MLMC
n_run_MLMC = 1

# initialize parameters for run
samples = 20
n_run = 40
time_seconds = 50
training_strategy = 'Random' # 'AL' or 'Random'

######################################################################################################
# Hyperparameters for active learning
n_initial = 365 * 2 # Initial training set, 5 years
n_queries = 20 # 10 years
temporal_pool_size = 3650 # 10 years
n_instances = 91 # 1 season = 91 days

################################################################################################
# Hyperparameters for random sampling
n_train_random = 365 * 5 + 365 * 5 * 3 # Initial training set, 5 years


# Estimator: Surr , Architecture: Exact|Surrogate models
ml_hierarchy = ['OptimalNStore', 'AIModel']

# [
#  "Training time", "Simulation time", "mean_LOL", "stderr_LOL", "speed_LOL", "mean_ENS", "stderr_ENS", "speed_ENS",
#  "Exact model calls", "exact model time per run", "Surrogate model calls", "Surrogate model time per run",
#  "exact_contribution_LOL_mean", "exact_contribution_LOL_std", "surrogate_contribution_LOL_mean", "surrogate_contribution_LOL_std",
#  "exact_contribution_ENS_mean", "exact_contribution_ENS_std", "surrogate_contribution_ENS_mean", "surrogate_contribution_ENS_std",
# ]
results = np.zeros((n_run_MLMC, 20))

print('Training strategy: ', training_strategy)
print('n_train_random: ', n_train_random)

for i in range(n_run_MLMC):
    lol_learner, ens_learner, results[i,0] = surrogate_model_training(system, training_strategy, n_train_random, n_initial, n_queries, temporal_pool_size, n_instances)
    system.AI_model.lol_model = lol_learner
    system.AI_model.ens_model = ens_learner

    mcc = run_MLMC(system, samples, n_run, time_seconds, ml_hierarchy, use_joblib=False, verbose=False)
    results[i,1:] = generate_mcc_results(mcc)

avg_results = np.mean(results, axis=0)
print('Average results: ', avg_results)

Training strategy:  Random
n_train_random:  7300
Running: {frozenset({'OptimalNStore', 'AIModel'}): 20, frozenset({'AIModel'}): 20}
relative sample numbers:  {frozenset({'OptimalNStore', 'AIModel'}): 463.86032069322016, frozenset({'AIModel'}): 18562.718093878564}
Running: {frozenset({'OptimalNStore', 'AIModel'}): 28, frozenset({'AIModel'}): 1127}
relative sample numbers:  {frozenset({'OptimalNStore', 'AIModel'}): 4149.616244039154, frozenset({'AIModel'}): 108925.64306890673}
Running: {frozenset({'OptimalNStore', 'AIModel'}): 31, frozenset({'AIModel'}): 819}
relative sample numbers:  {frozenset({'OptimalNStore', 'AIModel'}): 3298.8276266778093, frozenset({'AIModel'}): 88427.6621520735}
Running: {frozenset({'OptimalNStore', 'AIModel'}): 31, frozenset({'AIModel'}): 834}
relative sample numbers:  {frozenset({'OptimalNStore', 'AIModel'}): 2839.497690095018, frozenset({'AIModel'}): 77993.05717645385}
Running: {frozenset({'OptimalNStore', 'AIModel'}): 30, frozenset({'AIModel'}): 850}
relative

In [11]:
# Number of times to run MLMC
n_run_MLMC = 1

# initialize parameters for run
samples = 20
n_run = 40
time_seconds = 50
training_strategy = 'Random' # 'AL' or 'Random'

######################################################################################################
# Hyperparameters for active learning
n_initial = 365 * 2 # Initial training set, 5 years
n_queries = 20 # 10 years
temporal_pool_size = 3650 # 10 years
n_instances = 91 # 1 season = 91 days

################################################################################################
# Hyperparameters for random sampling
n_train_random = 365 * 5 + 365 * 5 * 7 # Initial training set, 5 years


# Estimator: Surr , Architecture: Exact|Surrogate models
ml_hierarchy = ['OptimalNStore', 'AIModel']

# [
#  "Training time", "Simulation time", "mean_LOL", "stderr_LOL", "speed_LOL", "mean_ENS", "stderr_ENS", "speed_ENS",
#  "Exact model calls", "exact model time per run", "Surrogate model calls", "Surrogate model time per run",
#  "exact_contribution_LOL_mean", "exact_contribution_LOL_std", "surrogate_contribution_LOL_mean", "surrogate_contribution_LOL_std",
#  "exact_contribution_ENS_mean", "exact_contribution_ENS_std", "surrogate_contribution_ENS_mean", "surrogate_contribution_ENS_std",
# ]
results = np.zeros((n_run_MLMC, 20))

print('Training strategy: ', training_strategy)
print('n_train_random: ', n_train_random)

for i in range(n_run_MLMC):
    lol_learner, ens_learner, results[i,0] = surrogate_model_training(system, training_strategy, n_train_random, n_initial, n_queries, temporal_pool_size, n_instances)
    system.AI_model.lol_model = lol_learner
    system.AI_model.ens_model = ens_learner

    mcc = run_MLMC(system, samples, n_run, time_seconds, ml_hierarchy, use_joblib=False, verbose=False)
    results[i,1:] = generate_mcc_results(mcc)

avg_results = np.mean(results, axis=0)
print('Average results: ', avg_results)

Training strategy:  Random
n_train_random:  14600
Running: {frozenset({'OptimalNStore', 'AIModel'}): 20, frozenset({'AIModel'}): 20}
relative sample numbers:  {frozenset({'OptimalNStore', 'AIModel'}): 1607.9739471432567, frozenset({'AIModel'}): 59958.45687153399}
Running: {frozenset({'OptimalNStore', 'AIModel'}): 28, frozenset({'AIModel'}): 1065}
relative sample numbers:  {frozenset({'OptimalNStore', 'AIModel'}): 1902.908459676918, frozenset({'AIModel'}): 61873.16155648366}
Running: {frozenset({'OptimalNStore', 'AIModel'}): 29, frozenset({'AIModel'}): 961}
relative sample numbers:  {frozenset({'OptimalNStore', 'AIModel'}): 2605.2434957428854, frozenset({'AIModel'}): 84623.1696378246}
Running: {frozenset({'OptimalNStore', 'AIModel'}): 29, frozenset({'AIModel'}): 962}
relative sample numbers:  {frozenset({'OptimalNStore', 'AIModel'}): 2251.356621863512, frozenset({'AIModel'}): 73111.64441988549}
Running: {frozenset({'OptimalNStore', 'AIModel'}): 29, frozenset({'AIModel'}): 963}
relative 

In [12]:
# Number of times to run MLMC
n_run_MLMC = 1

# initialize parameters for run
samples = 20
n_run = 40
time_seconds = 50
training_strategy = 'Random' # 'AL' or 'Random'

######################################################################################################
# Hyperparameters for active learning
n_initial = 365 * 2 # Initial training set, 5 years
n_queries = 20 # 10 years
temporal_pool_size = 3650 # 10 years
n_instances = 91 # 1 season = 91 days

################################################################################################
# Hyperparameters for random sampling
n_train_random = 365 * 5 + 365 * 4 * 1 # Initial training set, 5 years


# Estimator: Surr , Architecture: Exact|Surrogate models
ml_hierarchy = ['OptimalNStore', 'AIModel']

# [
#  "Training time", "Simulation time", "mean_LOL", "stderr_LOL", "speed_LOL", "mean_ENS", "stderr_ENS", "speed_ENS",
#  "Exact model calls", "exact model time per run", "Surrogate model calls", "Surrogate model time per run",
#  "exact_contribution_LOL_mean", "exact_contribution_LOL_std", "surrogate_contribution_LOL_mean", "surrogate_contribution_LOL_std",
#  "exact_contribution_ENS_mean", "exact_contribution_ENS_std", "surrogate_contribution_ENS_mean", "surrogate_contribution_ENS_std",
# ]
results = np.zeros((n_run_MLMC, 20))

print('Training strategy: ', training_strategy)
print('n_train_random: ', n_train_random)

for i in range(n_run_MLMC):
    lol_learner, ens_learner, results[i,0] = surrogate_model_training(system, training_strategy, n_train_random, n_initial, n_queries, temporal_pool_size, n_instances)
    system.AI_model.lol_model = lol_learner
    system.AI_model.ens_model = ens_learner

    mcc = run_MLMC(system, samples, n_run, time_seconds, ml_hierarchy, use_joblib=False, verbose=False)
    results[i,1:] = generate_mcc_results(mcc)

avg_results = np.mean(results, axis=0)
print('Average results: ', avg_results)

Training strategy:  Random
n_train_random:  3285
Running: {frozenset({'OptimalNStore', 'AIModel'}): 20, frozenset({'AIModel'}): 20}
relative sample numbers:  {frozenset({'OptimalNStore', 'AIModel'}): 1346.6261252525437, frozenset({'AIModel'}): 55170.76946409308}
Running: {frozenset({'OptimalNStore', 'AIModel'}): 27, frozenset({'AIModel'}): 1135}
relative sample numbers:  {frozenset({'OptimalNStore', 'AIModel'}): 2222.9940876630835, frozenset({'AIModel'}): 70617.5830587048}
Running: {frozenset({'OptimalNStore', 'AIModel'}): 29, frozenset({'AIModel'}): 944}
relative sample numbers:  {frozenset({'OptimalNStore', 'AIModel'}): 1835.8785512462296, frozenset({'AIModel'}): 59599.465087928336}
Running: {frozenset({'OptimalNStore', 'AIModel'}): 29, frozenset({'AIModel'}): 960}
relative sample numbers:  {frozenset({'OptimalNStore', 'AIModel'}): 1833.236815916746, frozenset({'AIModel'}): 60072.71401254603}
Running: {frozenset({'OptimalNStore', 'AIModel'}): 29, frozenset({'AIModel'}): 969}
relative

# Exact model

In [ ]:
# Number of times to run MLMC
n_run_MLMC = 10

# initialize parameters for run
samples = 25
n_run = 40
time_seconds = 50

# Estimator: MC , Architecture: Exact
ml_hierarchy = ['OptimalNStore']

# [
#  "Training time", "Simulation time", "mean_LOL", "stderr_LOL", "speed_LOL", "mean_ENS", "stderr_ENS", "speed_ENS",
#  "Exact model calls", "exact model time per run",
#  "exact_contribution_LOL_mean", "exact_contribution_LOL_std",
#  "exact_contribution_ENS_mean", "exact_contribution_ENS_std"
# ]
results = np.zeros((n_run_MLMC, 14))

for i in range(n_run_MLMC):
    time_start = time.time()
    mcc = run_MLMC(system, samples, n_run, time_seconds, ml_hierarchy, use_joblib=False, verbose=False)
    time_end = time.time()
    results[i,0] = time_end - time_start
    results[i,1:] = generate_mcc_results(mcc)

avg_results = np.mean(results, axis=0)
print('Average results: ', avg_results)